In [1]:
import tensorflow as tf
import random
import numpy as np
from keras import datasets, models, layers, optimizers
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import random

seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
train_images = x_train.astype('float32') / 255
test_images = x_test.astype('float32') / 255
train_labels = to_categorical(y_train)
test_labels = to_categorical(y_test)
val_images = train_images[:10000]
partial_images = train_images[10000:]
val_labels = train_labels[:10000]
partial_labels = train_labels[10000:]

number_of_epochs = 10
def create_and_train_model(params):
    num_layers = params[0]
    num_filters_params = params[1:num_layers + 1]
    kernel_sizes_params = params[num_layers + 1:2 * num_layers + 1]

    dropout_rate_cnn = 0.1
    dropout_rate_mlp = 0.5
    optimizer = optimizers.Adam()
    activation_function = 'relu'

    model = models.Sequential()
    input_shape = (32, 32, 3)
    num_classes = 10
    num_epochs = number_of_epochs

    model.add(layers.Conv2D(num_filters_params[0], (kernel_sizes_params[0], kernel_sizes_params[0]), padding='same', input_shape=input_shape, activation=activation_function))
    model.add(layers.BatchNormalization())

    for i in range(1, num_layers):
        kernel_size = kernel_sizes_params[i]
        if kernel_size > model.output_shape[1]:
            kernel_size = model.output_shape[1]

        model.add(layers.Conv2D(num_filters_params[i], (kernel_size, kernel_size), padding='same', activation=activation_function))
        model.add(layers.BatchNormalization())

        if i % 2 == 1 and model.output_shape[1] > 2 and model.output_shape[2] > 2:
            model.add(layers.MaxPooling2D(2, 2))

    if model.output_shape[1] <= 0 or model.output_shape[2] <= 0:
        raise ValueError(f"Output dimensions are invalid: {model.output_shape}")

    model.add(layers.Flatten())
    model.add(layers.Dropout(dropout_rate_cnn))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(dropout_rate_mlp))
    model.add(layers.Dense(num_classes, activation="softmax"))

    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

    es = EarlyStopping(monitor="val_accuracy", patience=3)
    history = model.fit(partial_images, partial_labels, validation_data=(val_images, val_labels), epochs=num_epochs, callbacks=[es], verbose=0)

    return params, history.history['val_accuracy'][-1]

170498071/170498071 [==============================] - 13s 0us/step


In [2]:
def generate_binary_list(length):
    return [random.randint(0, 1) for _ in range(length)]
def one_point_crossover(parent1, parent2):
    # انتخاب تصادفی نقطه‌ی pivot از 1 تا طول کوچکترین والد - 1
    pivot = random.randint(1, min(len(parent1), len(parent2)) - 1)

    # انجام فرآیند cross over بدون در نظر گرفتن ژن اول
    offspring1 = parent1[:pivot] + parent2[pivot:]
    offspring2 = parent2[:pivot] + parent1[pivot:]

    # تنظیم ژن اول هر فرزند
    offspring1[0] = len(offspring1[1:]) // 2
    offspring2[0] = len(offspring2[1:]) // 2

    return offspring1, offspring2


def crossover(parent1, parent2, current_generation, total_generations):
    prob_binary_crossover = 1 - current_generation / total_generations
    prob_one_point_crossover = 1 - prob_binary_crossover

    print(parent1 , parent2)

    if random.random() < prob_binary_crossover:
        n = len(parent1)
        m = len(parent2)
        binary_list = generate_binary_list(max(n, m) - 1)

        offspring1 = [parent1[0]]
        offspring2 = [parent2[0]]

        for i in range(1, min(n, m)):
            if binary_list[i-1] == 0:
                offspring1.append(parent1[i])
                offspring2.append(parent2[i])
            else:
                offspring1.append(parent2[i])
                offspring2.append(parent1[i])

        if n > m:
            offspring1.extend(parent1[m:])
        elif m > n:
            offspring2.extend(parent2[n:])

    else:
        offspring1, offspring2 = one_point_crossover(parent1, parent2)

    return offspring1, offspring2

def mutate_individual(individual, mutation_rate=0.3):
    if random.random() < mutation_rate:
        mutation_type = random.choice(['allele', 'layers'])
        if mutation_type == 'allele':
            idx = random.randint(1, len(individual) - 1)
            if idx % 2 == 0:
                individual[idx] = random.randint(2, 9)
            else:
                individual[idx] = random.randint(4, 128)
        else:
            new_n = random.randint(2, 7)
            old_n = individual[0]
            individual[0] = new_n
            if new_n < old_n:
                individual = individual[:-(2 * (old_n - new_n))]
            elif new_n > old_n:
                additional_genes = []
                for _ in range(2 * (new_n - old_n)):
                    if len(additional_genes) % 2 == 0:
                        additional_genes.append(random.randint(4, 128))
                    else:
                        additional_genes.append(random.randint(2, 9))
                individual.extend(additional_genes)
    return individual

def tournament_selection(population, scores, k=2):
    selected = random.sample(list(zip(population, scores)), k)
    selected.sort(key=lambda x: x[1], reverse=True)
    return selected[0][0]

def create_children(population, scores, number_of_children, current_generation, total_generations):
    number_of_children = number_of_children // 2

    for _ in range(number_of_children):
        parent1 = tournament_selection(population, scores)
        parent2 = tournament_selection(population, scores)

        child1, child2 = crossover(parent1, parent2, current_generation, total_generations)

        child1 = mutate_individual(child1)
        child2 = mutate_individual(child2)

        try:
            a, b = create_and_train_model(child1)
            population.append(child1)
            scores.append(b)
        except Exception as e:
            print(f"Error training child1: {e}")
            continue

        try:
            c, d = create_and_train_model(child2)
            population.append(child2)
            scores.append(d)
        except Exception as e:
            print(f"Error training child2: {e}")
            continue


def rank_based_selection(population, scores, num_to_keep):
    ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

    # Convert ranks to probabilities
    total_rank = sum(range(1, len(ranked_indices) + 1))
    probabilities = [(len(ranked_indices) - rank) / total_rank for rank in range(len(ranked_indices))]

    # Select indices based on their probabilities without replacement
    selected_indices = set()
    while len(selected_indices) < num_to_keep:
        selected_index = random.choices(ranked_indices, weights=probabilities, k=1)[0]
        selected_indices.add(selected_index)

    # Create new population and scores based on the selected indices
    selected_indices = list(selected_indices)
    population[:] = [population[i] for i in selected_indices]
    scores[:] = [scores[i] for i in selected_indices]


def fitness_based_selection(population, scores, num_to_keep):
    # Normalize the scores to probabilities
    total_fitness = sum(scores)
    probabilities = [score / total_fitness for score in scores]

    # Select indices based on their probabilities without replacement
    selected_indices = set()
    while len(selected_indices) < num_to_keep:
        selected_index = random.choices(range(len(population)), weights=probabilities, k=1)[0]
        selected_indices.add(selected_index)

    # Create new population and scores based on the selected indices
    selected_indices = list(selected_indices)
    population[:] = [population[i] for i in selected_indices]
    scores[:] = [scores[i] for i in selected_indices]



def survivor_selection(population, scores, num_survivors, generation, total_generations):
    probability_fitness_based = 1 - generation / total_generations
    probability_rank_based = 1 - probability_fitness_based

    if random.random() < probability_fitness_based:
        fitness_based_selection(population, scores, num_survivors)
    else:
        rank_based_selection(population, scores, num_survivors)


In [ ]:
# تنظیمات الگوریتم تکاملی
population_size = 12
generations = 12
number_of_children = 6

# # تولید هایپرپارامترهای تصادفی اولیه
# def generate_random_hyperparameters():
#     first_gene = random.randint(2, 7)
#     chromosome_length = 1 + 2 * first_gene
#     chromosome = [first_gene]
#     for i in range(1, chromosome_length):
#         if i % 2 == 0:
#             chromosome.append(random.randint(2, 9))
#         else:
#             chromosome.append(random.randint(4, 128))
    # return chromosome

# ایجاد جمعیت اولیه
population = [[4, 110, 4, 98, 3, 17, 3, 21, 3], [7, 38, 2, 31, 7, 35, 6, 52, 3, 90, 3, 79, 8, 8, 2], [4, 110, 2, 98, 3, 17, 3, 21, 3], [7, 38, 4, 31, 7, 35, 6, 52, 3, 90, 3, 79, 8, 114, 2], [7, 39, 4, 31, 7, 17, 6, 52, 3, 90, 3, 79, 8, 114, 2], [4, 38, 4, 31, 7, 35, 6, 52, 3], [4, 39, 4, 31, 7, 17, 6, 52, 3], [5, 38, 9, 31, 7, 31, 9, 54, 9, 22, 6], [5, 38, 9, 31, 7, 31, 9, 54, 9, 22, 6], [7, 18, 2, 98, 6, 35, 6, 52, 3, 90, 3, 79, 8, 114, 2], [7, 39, 4, 31, 7, 17, 6, 52, 3, 90, 3, 79, 8, 114, 2], [7, 38, 4, 31, 7, 35, 6, 52, 3, 90, 3, 79, 8, 114, 2]]
# scores = [0.5034999847412109, 0.49559998512268066, 0.4973999857902527, 0.5479000210762024, 0.5863000154495239, 0.4544000029563904, 0.5288000106811523, 0.5199999809265137, 0.5163999795913696, 0.5472999811172485, 0.5631999969482422, 0.5785999894142151]
scores = []

for individual in population:
    try:
        _, accuracy = create_and_train_model(individual)
        scores.append(accuracy)
    except Exception as e:
        print(f"Error training individual: {e}")
        scores.append(0)


for i in range(generations) :
  create_children(population, scores, number_of_children, i , generations)
  survivor_selection(population, scores, population_size , i , generations)
  print(i + 1  , '/' , generations)
  print('populations : ' , population )
  print('Accuracy : ' , scores)
  print('mean : ' , np.mean(scores))
  print('best : ' , np.max(scores))

[5, 38, 9, 31, 7, 31, 9, 54, 9, 22, 6] [4, 110, 2, 98, 3, 17, 3, 21, 3]
[7, 38, 4, 31, 7, 35, 6, 52, 3, 90, 3, 79, 8, 114, 2] [7, 38, 4, 31, 7, 35, 6, 52, 3, 90, 3, 79, 8, 114, 2]
[5, 38, 9, 31, 7, 31, 9, 54, 9, 22, 6] [7, 38, 2, 31, 7, 35, 6, 52, 3, 90, 3, 79, 8, 8, 2]
1 / 12
populations :  [[4, 110, 4, 98, 3, 17, 3, 21, 3], [7, 38, 2, 31, 7, 35, 6, 52, 3, 90, 3, 79, 8, 8, 2], [7, 39, 4, 31, 7, 17, 6, 52, 3, 90, 3, 79, 8, 114, 2], [4, 39, 4, 31, 7, 17, 6, 52, 3], [5, 38, 9, 31, 7, 31, 9, 54, 9, 22, 6], [5, 38, 9, 31, 7, 31, 9, 54, 9, 22, 6], [7, 18, 2, 98, 6, 35, 6, 52, 3, 90, 3, 79, 8, 114, 2], [7, 39, 4, 31, 7, 17, 6, 52, 3, 90, 3, 79, 8, 114, 2], [5, 38, 2, 31, 7, 31, 9, 21, 3, 22, 6], [4, 110, 9, 98, 3, 17, 3, 54, 9], [4, 38, 2, 31, 7, 31, 9, 52, 9], [7, 38, 9, 31, 7, 35, 6, 54, 3, 90, 3, 79, 8, 8, 2]]
Accuracy :  [0.6018000245094299, 0.6103000044822693, 0.6478000283241272, 0.6146000027656555, 0.6194000244140625, 0.6072999835014343, 0.5956000089645386, 0.5741000175476074, 0.601999